In [36]:
from shutil import copytree, ignore_patterns, copy
from distutils.dir_util import copy_tree
import os
import numpy as np
import time
#from multiprocessing import Pool
import matplotlib.pyplot as plt
%matplotlib inline
from util_func import *

In [ ]:
def list_of_data(data_dir):
    list_of_lidar = []
    list_of_gtbox = []
    for f in os.listdir(data_dir):
        path = os.path.join(data_dir, f)
        lidar_path = os.path.join(path, 'lidar')
        gtbox_path = os.path.join(path, 'gt_boxes3d')
        num_files = len(os.listdir(lidar_path))

        lidar = [os.path.join(lidar_path, 'lidar_'+str(i)+'.npy') for i in range(num_files) ]
        gtbox = [os.path.join(gtbox_path, 'gt_boxes3d_'+str(i)+'.npy') for i in range(num_files) ]
        list_of_lidar += lidar
        list_of_gtbox += gtbox
    return list_of_lidar, list_of_gtbox

In [ ]:
data_dir = './all_data/extract_kiti/'
list_of_lidar, list_of_gtbox = list_of_data(data_dir)
print(len(list_of_lidar))

In [ ]:
num_cars = np.zeros(len(list_of_gtbox))
for i in range(len(list_of_gtbox)):
    gt_box = np.load(list_of_gtbox[i])
    num_cars[i] = len(gt_box)

In [ ]:
unique, counts = np.unique(num_cars, return_counts=True)
dict(zip(unique, counts))

In [ ]:
indx = {}
for i in range(len(num_cars)):
    if num_cars[i] not in indx.keys():
        indx[num_cars[i]] = [i]
    else:
        indx[num_cars[i]].append(i)
print(indx.keys())

In [ ]:
import pickle

with open('./saved_model/numcar_ind.pickle', 'wb') as f:
    pickle.dump(indx, f)

with open('./saved_model/numcar_ind.pickle', 'rb') as f:
    ind_load = pickle.load(f)

print(indx == ind_load)

In [ ]:
print(int(0.1*len(ind_load[0])))

In [ ]:
def undersample_data(list_of_lidar, list_of_gtbox, pickle_index, percent_noncar = 0.1):
    with open(pickle_index, 'rb') as f:
        ind_load = pickle.load(f)
    size_noncar = int(0.1*len(ind_load[0]))
    ind_0 = list(np.random.choice(ind_load[0], size=size_noncar, replace=False))
    for i in range(1, len(ind_load)):
        ind_0 += ind_load[i]
    new_list_of_lidar = [list_of_lidar[i] for i in ind_0]
    new_list_of_gtbox = [list_of_gtbox[i] for i in ind_0]
   
    return new_list_of_lidar, new_list_of_gtbox

pickle_index = './saved_model/numcar_ind.pickle'
new_list_of_lidar, new_list_of_gtbox = undersample_data(list_of_lidar, list_of_gtbox, 
                                                       pickle_index, percent_noncar = 0.1)
print(len(new_list_of_lidar), len(new_list_of_gtbox))

In [ ]:
for i in indx.keys():
    print(i,' : ',np.random.choice(indx[i]), len(indx[i]) )

In [40]:
# This extracts view folder from the rest of the dataset

# set input as extract_kiti
# set out as desired directory
directory="/home/hanbin/Desktop/Udacity_challenge/our_didi_v2/didi_challenge/extract_kiti_real"
dst="/home/hanbin/Desktop/Udacity_challenge/our_didi_v2/didi_challenge/view_data"

src=directory
print src
print dst

copytree(src, dst, ignore=ignore_patterns('lidar*', 'gt_boxes3d*', 'gt_labels*'))

/home/hanbin/Desktop/Udacity_challenge/our_didi_v2/didi_challenge/extract_kiti_real
/home/hanbin/Desktop/Udacity_challenge/our_didi_v2/didi_challenge/view_data
